In [29]:
import pandas as pd

df = pd.read_csv('test.csv')

In [25]:
import string
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords

def preprocesar(c):

    wnl = WordNetLemmatizer()
    ps = PorterStemmer()
    for i in range(len(c)):
        s = str(c[i])
        s = s.lower()
        s = s.translate(str.maketrans('', '', string.punctuation))
        s = s.split()
        stop_words=set(stopwords.words('english'))
        filtered_sentence = [w for w in s if not w in stop_words]
        filtered_sentence = []
        for w in s:
            if w not in stop_words and len(w)>1:
                w = wnl.lemmatize(w)
                w = ps.stem(w)
                filtered_sentence.append(w)
        s = filtered_sentence
        c[i] = s
    return(c)    
    

In [30]:
df['title'] = preprocesar(df['title'])

/tmp/ipykernel_15964/4127489850.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[i] = s


In [27]:
df['text'] = preprocesar(df['text'])

/tmp/ipykernel_15964/4127489850.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[i] = s


In [28]:
df.to_csv('test_p.csv', index=False)